# wiki40bをデコードして使う方法

## 参考リンク
- https://www.tensorflow.org/datasets/catalog/wiki40b
- https://note.com/npaka/n/n8b1b87b45162
- https://www.yurui-deep-learning.com/2021/03/02/hugging-face-japanese/

## wiki40bのフォーマット
- 記事ごとに、(_START_ARTICLE_ )、（_START_SECTION_）、（_START_PARAGRAPH_）を区切りとした構成
- (_START_PARAGRAPH_)では、(_NEWLINE_)がセクション内のパラグラフの終わりとして挿入されている
- (_START_SECTION_）と（_START_PARAGRAPH_）は一致しない


### 例）
_START_ARTICLE_  
南部煎餅  
_START_SECTION_  
概要  
_START_PARAGRAPH_  
元々は八戸藩で作られた非常食であり、小麦粉を水で練って円形の型に入れて堅く焼いて作られる。縁に「みみ」と呼ばれる薄くカリッとした部分があるのが特徴。保存性は非常によいが、時間が経過すると酸化により味が落ちる。個包装の商品も存在するが、通常は10 - 20枚程度を1つの袋に入れた簡素なものが多い。青森、岩手の旧南部氏支配地域においては非常にポピュラーな食べ物であり、来客にも供される。   
_START_SECTION_  
種類  
_START_PARAGRAPH_  
通常の「白せんべい」と呼ばれるものの他にゴマ、クルミ、落花生などを加えて焼いたものもある。また同じ材料で厚めに焼き、食感を柔らかく仕上げた「てんぽせんべい」がある。_NEWLINE_近年ではイカ、カボチャ、リンゴ、ココアなどバリエーションが豊富である。クッキー状の生地で作られるものもある。ただし、通常スーパー等で売っているものと言えばまずゴマ、次いで落花生の二種類であり、他の種類のものはメーカー直営店や土産物屋、南部煎餅専門コーナー等以外では入手しにくい。地域によっても味が微妙に異なる。青森県で消費される南部煎餅は比較的薄くてほんのり塩味があり、岩手県で消費される南部煎餅は少し厚みがありほんのり甘い傾向がある。  
_START_SECTION_  
食べ方  
_START_PARAGRAPH_  
そのまま食べるのが一般的であるが、その他に、水飴や赤飯を挟んで食すこともある（せんべいおこわを参照）。水飴を挟んだものは「飴せん」と呼ばれ、津軽地方の「津軽飴」を用いることが多い。_NEWLINE_また、パン代わりに「白せんべい」または「ゴマせんべい」をトースターで加熱し、バター等を塗って食べる人もいる。  
_START_SECTION_  
区域  
_START_PARAGRAPH_  
現在は岩手県南部煎餅協同組合が「南部せんべい」の名で商標登録しているが、実際は南部藩の領域で主に生産されてきたものであり、青森県南部・岩手県・北海道にも存在しており広範囲にわたっている。岩手県でも、旧仙台藩にあたる岩手県南部でも生産・消費されている。  
_START_SECTION_  
明治・大正期  
_START_PARAGRAPH_  
[...]  

## コード

### パッケージと関数

In [2]:
# パッケージ

import tensorflow_datasets as tfds

import pandas as pd
import pickle

In [3]:
# ロードしたファイルをデコードする関数

def decode_wiki40b(ds, outname, is_dump=True):
    articles = []
    for wiki in ds.as_numpy_iterator():
        lines = wiki['text'].decode().split('\n')
        paragraphs = []        
        for i in range(len(lines)):
            if lines[i] == '_START_ARTICLE_':            
                title = lines[i+1].replace('_NEWLINE_', '')
            if lines[i] == '_START_PARAGRAPH_':
                paragraphs.append(lines[i+1].replace('_NEWLINE_', ''))
        articles.append([title, paragraphs])

    if is_dump:
        with open(outname, 'wb') as f:
            pickle.dump(articles, f)
        
    return articles

# ファイル出力したpickleファイルを読み込むとき
# with open(outname, 'rb') as f:
#     pickle.load(f)

### ファイルのロード

In [4]:
# wikiデータのロード

# 'test'	41,268
# 'train'	745,392
# 'validation'	41,576

In [5]:
ds_test, dsinfo_test = tfds.load('wiki40b/ja', split='test', with_info=True, try_gcs=True)

In [6]:
ds_train, dsinfo_train = tfds.load('wiki40b/ja', split='train', with_info=True, try_gcs=True)

In [7]:
ds_validation, dsinfo_validation = tfds.load('wiki40b/ja', split='validation', with_info=True, try_gcs=True)

### ファイルのdecode

In [8]:
airticles_test = decode_wiki40b(ds_test, 'wiki_40b_test.pkl')

In [9]:
airticles_train = decode_wiki40b(ds_train, 'wiki_40b_train.pkl')

In [10]:
airticles_val = decode_wiki40b(ds_validation, 'wiki_40b_val.pkl')

### 中身の確認

In [11]:
len(airticles_test), len(airticles_train), len(airticles_val)

(41268, 745392, 41576)

In [25]:
airticles_test[3]

['独孤羅',
 ['独孤 羅（どっこ ら、534年 - 599年）は、北魏末から隋にかけての人物。字は羅仁。本貫は雲中郡。',
  '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤信が宇文護により処刑されると、ようやく釈放されて、中山に寓居した。北斉の独孤永業の一族として田宅を与えられた。北斉が滅亡し、楊堅が定州総管となると、楊堅の妻の独孤伽羅が兄の行方を捜索させて独孤羅を見つけ出し、初めて対面した。579年（大象元年）、功臣の子として楚安郡太守に任じられた。まもなく病のため辞職し、長安に帰った。580年（大象2年）、楊堅が北周の丞相となると、独孤羅は儀同大将軍の位を受け、楊堅の側近に仕えた。581年（開皇元年）、隋が建国されると、使持節・上開府・儀同大将軍の位を受けた。11月、右武衛将軍に転じた。582年（開皇2年）、父の趙国公の爵位を嗣いだ。592年（開皇12年）、大将軍・太子右衛率となった。593年（開皇13年）、涼州刺史に任じられた。597年（開皇17年）、涼州総管に任じられた。599年（開皇19年）2月6日、死去した。諡は徳といった。']]

In [26]:
airticles_test[3][0]

'独孤羅'

In [27]:
airticles_test[3][1]

['独孤 羅（どっこ ら、534年 - 599年）は、北魏末から隋にかけての人物。字は羅仁。本貫は雲中郡。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤信が宇文護により処刑されると、ようやく釈放されて、中山に寓居した。北斉の独孤永業の一族として田宅を与えられた。北斉が滅亡し、楊堅が定州総管となると、楊堅の妻の独孤伽羅が兄の行方を捜索させて独孤羅を見つけ出し、初めて対面した。579年（大象元年）、功臣の子として楚安郡太守に任じられた。まもなく病のため辞職し、長安に帰った。580年（大象2年）、楊堅が北周の丞相となると、独孤羅は儀同大将軍の位を受け、楊堅の側近に仕えた。581年（開皇元年）、隋が建国されると、使持節・上開府・儀同大将軍の位を受けた。11月、右武衛将軍に転じた。582年（開皇2年）、父の趙国公の爵位を嗣いだ。592年（開皇12年）、大将軍・太子右衛率となった。593年（開皇13年）、涼州刺史に任じられた。597年（開皇17年）、涼州総管に任じられた。599年（開皇19年）2月6日、死去した。諡は徳といった。']

In [ ]:
# 記事ごとにパラグラフを結合して１つの文章にして使うとき

In [39]:
airticles_test2 = [[airticle[0], "".join(airticle[1])] for airticle in airticles_test]

In [43]:
airticles_test2[3]

['独孤羅',
 '独孤 羅（どっこ ら、534年 - 599年）は、北魏末から隋にかけての人物。字は羅仁。本貫は雲中郡。534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤信が宇文護により処刑されると、ようやく釈放されて、中山に寓居した。北斉の独孤永業の一族として田宅を与えられた。北斉が滅亡し、楊堅が定州総管となると、楊堅の妻の独孤伽羅が兄の行方を捜索させて独孤羅を見つけ出し、初めて対面した。579年（大象元年）、功臣の子として楚安郡太守に任じられた。まもなく病のため辞職し、長安に帰った。580年（大象2年）、楊堅が北周の丞相となると、独孤羅は儀同大将軍の位を受け、楊堅の側近に仕えた。581年（開皇元年）、隋が建国されると、使持節・上開府・儀同大将軍の位を受けた。11月、右武衛将軍に転じた。582年（開皇2年）、父の趙国公の爵位を嗣いだ。592年（開皇12年）、大将軍・太子右衛率となった。593年（開皇13年）、涼州刺史に任じられた。597年（開皇17年）、涼州総管に任じられた。599年（開皇19年）2月6日、死去した。諡は徳といった。']

In [ ]:
# 同一記事内の_START_SECTION_と_START_SECTION_の確認

In [13]:
ds = ds_test

articles_section_paragraph = []
for wiki in ds.as_numpy_iterator():
    lines = wiki['text'].decode().split('\n')
    counts = [0, 0]
    for i in range(len(lines)):
        if lines[i] == '_START_SECTION_':
            counts[0] += 1
        if lines[i] == '_START_PARAGRAPH_':
            counts[1] += 1
    articles_section_paragraph.append(counts)

In [15]:
df = pd.DataFrame(articles_section_paragraph)
df.head()

,0,1
0,1,1
1,2,2
2,1,1
3,1,2
4,1,2


In [18]:
(df[0] == df[1]).sum() / len(df)

0.676480566056024